In [46]:
from API.models import *
restaurant=Restaurant.objects.filter(id=5)
reviewsByUser=User.objects.get(id=18)
print(reviewsByUser.id)
#c=RecommendedRate(user_id=1, restaurant_id=1,rating=5)
#c.rating
#reviewsByUser.RecommendedRate
#gAll=gathering.objects.all()
rr=reviewsByUser.recommendedrate_set.all().order_by('-rating')
#posts_lists = map(lambda post: project.post, rr)
#print(posts_lists.values())
mkey=list(rr.values_list('gathering_id',flat=True))
print(mkey)
clauses = ' '.join(['WHEN id=%s THEN %s' % (pk, i) for i, pk in enumerate(mkey)])
ordering = 'CASE %s END' % clauses
queryset = Gathering.objects.filter(pk__in=mkey).extra(
           select={'ordering': ordering}, order_by=('ordering',))
print(queryset)

Gathering.objects.filter(id__in=rr.values('gathering_id'))
#g=Gathering.objects.get(id=27)
#r=g.recommend.get(id=207)
#r.gathering.id

18
[19, 20, 21, 25, 28, 22, 26, 27]
<QuerySet [<Gathering: binno together!>, <Gathering: Delicious!>, <Gathering: First gathering>, <Gathering: qfgggfffdfffff>, <Gathering: uhjkkkkkj>, <Gathering: Tesing>, <Gathering: gsjsbjsjs>, <Gathering: gugjjjj>]>


<QuerySet [<Gathering: binno together!>, <Gathering: Delicious!>, <Gathering: First gathering>, <Gathering: Tesing>, <Gathering: qfgggfffdfffff>, <Gathering: gsjsbjsjs>, <Gathering: gugjjjj>, <Gathering: uhjkkkkkj>]>

In [47]:
reviewsByUser=Review.objects.filter(user=13)
for review in reviewsByUser:
	otherReviews=review.restaurant.review_set.exclude(id=review.id)
	for otherReview in otherReviews:
		print(review.rating-otherReview.rating)
	print('diff')		

-1
-2
-2
diff
3
diff


In [43]:
import pandas as pd
import numpy as np

userid=list(User.objects.all().order_by('id').values_list('id', flat=True))
restaurantid=list(Restaurant.objects.all().values_list('id', flat=True))

rating = pd.DataFrame(index=userid,columns=restaurantid)

allReviews=Review.objects.all()
for review in allReviews:
    rating.at[review.user_id,review.restaurant_id]=review.rating

print(rating)
print(np.isnan(rating.loc[13,5]))

      1    2    3    4    5    6    7
1   NaN  NaN  NaN  NaN  NaN  NaN  NaN
2   NaN  NaN  NaN  NaN  NaN  NaN  NaN
3   NaN  NaN  NaN  NaN  NaN  NaN  NaN
4   NaN  NaN  NaN  NaN  NaN  NaN  NaN
5   NaN  NaN  NaN  NaN  NaN  NaN  NaN
6   NaN  NaN  NaN  NaN  NaN  NaN  NaN
7   NaN  NaN  NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN  NaN  NaN
9   NaN  NaN  NaN  NaN  NaN  NaN  NaN
10  NaN  NaN  NaN  NaN  NaN  NaN  NaN
11  NaN  NaN  NaN  NaN    5    3    2
12  NaN  NaN  NaN  NaN    3    4  NaN
13  NaN  NaN  NaN  NaN  NaN    2    5
14  NaN  NaN  NaN  NaN  NaN  NaN  NaN
15  NaN  NaN  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN  NaN  NaN
17  NaN  NaN  NaN  NaN  NaN  NaN  NaN
18  NaN  NaN    2  NaN    4    4  NaN
True


In [53]:
diff = pd.DataFrame(index=restaurantid,columns=restaurantid)
freq = pd.DataFrame(index=restaurantid,columns=restaurantid)
matrixLenght=len(restaurantid)
for i in range(0,matrixLenght-1):
    for j in range(i+1,matrixLenght):
        ratingIndex1=restaurantid[i]
        ratingIndex2=restaurantid[j]
        temp=rating[ratingIndex1].sub(rating[ratingIndex2])
        count=temp.count()
        sum=temp.sum()
        if(count==0):
            diff.at[ratingIndex1,ratingIndex2]=0
            diff.at[ratingIndex2,ratingIndex1]=0
        else:
            diff.at[ratingIndex1,ratingIndex2]=sum/count
            diff.at[ratingIndex2,ratingIndex1]=-sum/count
        freq.at[ratingIndex1,ratingIndex2]=count
        freq.at[ratingIndex2,ratingIndex1]=count

print(diff)
print(freq)
print(diff.loc[1,1]/0)

#print(diff.iloc[1])

     1    2    3    4         5         6    7
1  NaN    0    0    0         0         0    0
2    0  NaN    0    0         0         0    0
3    0    0  NaN    0        -2        -2    0
4    0    0    0  NaN         0         0    0
5    0    0    2    0       NaN  0.333333    3
6    0    0    2    0 -0.333333       NaN   -1
7    0    0    0    0        -3         1  NaN
     1    2    3    4    5    6    7
1  NaN    0    0    0    0    0    0
2    0  NaN    0    0    0    0    0
3    0    0  NaN    0    1    1    0
4    0    0    0  NaN    0    0    0
5    0    0    1    0  NaN    3    1
6    0    0    1    0    3  NaN    2
7    0    0    0    0    1    2  NaN


ZeroDivisionError: float division by zero

In [59]:
predid=18
restid=7
notnullindex=rating.loc[predid,:].dropna().index
#print(notnullindex)
length=len(notnullindex)
ratingSum=0
freqSum=0
for i in range(0,length):
    ratingSum+=(rating.loc[predid,notnullindex[i]]+diff.loc[restid][notnullindex[i]])*freq.loc[restid][notnullindex[i]]
    freqSum+=freq.loc[restid][notnullindex[i]]
if(freqSum!=0):
    print(ratingSum/freqSum)
else:
    print(0)
#predictedrate=RecommendedRate(user_id=predid, restaurant_id=restid,rating=ratingSum/freqSum)
#predictedrate.save()


3.6666666666666665


In [196]:
import numpy as np

rating.loc[:,10] = np.nan
rating.loc[14,:]= np.nan
rating.loc[15,11]= 4
rating


,5,6,7,10,11
11,5,3,2,NaN,NaN
12,3,4,NaN,NaN,NaN
13,NaN,2,5,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,4.0


In [8]:

newReviewRest=5
freq.loc[newReviewRest,:] = freq.loc[newReviewRest,:] + 1
freq.loc[:,newReviewRest] = freq.loc[:,newReviewRest] + 1
freq

,5,6,7
5,NaN,3,2
6,3,NaN,2
7,2,2,NaN


In [208]:
restaurantid
matrixLenght=len(restaurantid)
for i in range(0,matrixLenght-1):
    ratingIndex1=restaurantid[i]
    ratingIndex2=restaurantid[j]
    temp=rating[ratingIndex1].sub(rating[ratingIndex2])
    count=temp.count()
    sum=temp.sum()
    diff.at[ratingIndex1,ratingIndex2]=sum/count
    diff.at[ratingIndex2,ratingIndex1]=-sum/count


[5, 6, 7]

In [11]:
RecommendedRate.objects.all()

NameError: name 'RecommendedRate' is not defined

In [5]:
from API.recommendation import *
s=SlopeOne()
s.predict(12,5)

nan


nan